### Disaster Tweets
#### `Hugo Hersemeule, Tom Kubasik`

In [1]:
import pandas

train_data = pandas.read_csv("./data/train.csv")
test_data = pandas.read_csv("./data/test.csv")

In [2]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

count_vectorizer = feature_extraction.text.CountVectorizer()

train_vectors = count_vectorizer.fit_transform(train_data["text"])
test_vectors = count_vectorizer.transform(test_data["text"])

In [3]:
clf = linear_model.RidgeClassifier()
scores = model_selection.cross_val_score(clf, train_vectors, train_data["target"], cv=5, scoring="f1")
scores

array([0.6025641 , 0.50126582, 0.56985004, 0.50781969, 0.67275495])

In [4]:
clf.fit(train_vectors, train_data["target"])

RidgeClassifier()

In [5]:
sample_submission = pandas.read_csv("./data/sample_submission.csv")
sample_submission["target"] = clf.predict(test_vectors)

In [6]:
sample_submission.to_csv("submission.csv", index=False)